In [11]:
from pathlib import Path
from label_studio_sdk.client import LabelStudio
from fishsense_api_sdk.client import Client
from label_studio_sdk import LseUserApi
from fishsense_api_sdk.models.user import User
from tqdm.notebook import tqdm
import asyncio

In [3]:
URL = "http://localhost:8000"

In [4]:
label_studio_api_key = Path("..") / ".label_studio_api_key"

label_studio_api_key.exists()

True

In [5]:
ls = LabelStudio(base_url=f"https://labeler.e4e.ucsd.edu", api_key=label_studio_api_key.read_text().strip())

In [6]:
users = list(ls.users.list())

len(users), users[0]

(137,
 LseUserApi(active_organization=1, active_organization_meta={'title': 'Label Studio', 'email': 'e4e@ucsd.edu'}, allow_newsletters=None, avatar=None, custom_hotkeys={}, date_joined=datetime.datetime(2024, 11, 23, 2, 38, 19, 683173, tzinfo=datetime.timezone.utc), email='e4e@ucsd.edu', first_name='', id=1, initials='e4', last_activity=datetime.datetime(2024, 11, 23, 2, 38, 19, 683140, tzinfo=datetime.timezone.utc), last_name='', org_membership=None, organization_membership=None, phone='', username=''))

In [7]:
async with Client(URL) as fs:
    fs_users = await asyncio.gather(*(fs.users.get(email=user.email) for user in users))
    fs_users = [user for user in fs_users if user is not None]

len(fs_users), fs_users[0]

(136,
 User(id=48, label_studio_id=1, email='e4e@ucsd.edu', first_name='', last_name='', last_activity=datetime.datetime(2024, 11, 23, 2, 38, 19, 683140, tzinfo=TzInfo(0)), date_joined=datetime.datetime(2024, 11, 23, 2, 38, 19, 683173, tzinfo=TzInfo(0))))

In [8]:
def from_label_studio(user: LseUserApi) -> User:
    """Create a User instance from a Label Studio user."""

    return User(
        id = None,
        label_studio_id=user.id,
        email=user.email,
        first_name=user.first_name,
        last_name=user.last_name,
        last_activity=user.last_activity,
        date_joined=user.date_joined,
    )

In [12]:
async with Client(URL) as fs:
    for user in tqdm(users):
        fs_user = await fs.users.get(email=user.email)
        if fs_user is None:
            await fs.users.post(from_label_studio(user))
        else:
            new_fs_user = from_label_studio(user)
            new_fs_user.id = fs_user.id

            await fs.users.put(new_fs_user)

  0%|          | 0/137 [00:00<?, ?it/s]